# Richards equation

In this tutorial we present how to solve a Richards equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=\Omega_1 \cup \Omega_2$, with $\Omega_1=(0,1)\times(0, 1/4)$ and $\Omega_2=(0,1)\times(1/4,1)$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, \psi)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:

$$
\psi(0,x,z)=
\left\{
\begin{array}{ll}
\begin{array}{l} 
-z+1/4, (x,z) \in \Omega_1\\
-3, (x,z) \in \Omega_2
\end{array}
\end{array}
\right., 
\qquad \nu \cdot q = 0 \text{ on } \Gamma_N,
\qquad \psi(t,x,z) = -4 \text{ on } \Gamma_D,
\qquad f = 
\left\{
\begin{array}{ll}
\begin{array}{l} 
0, (x,z) \in \Omega_1\\
0.006 \cos(\frac{4}{3}\pi (z-1)) \sin(2\pi x), (x,z) \in \Omega_2
\end{array}
\end{array}
\right.
$$
and
$$
\Gamma_{D} = (0,1) \times \{1\},\\
\Gamma_{N} = \partial \Omega \setminus \Gamma_D
$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

### Import and parameters

In [1]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
from scipy.sparse import linalg
import sympy as sp

import porepy as pp
import pygeon as pg

import time
from math import ceil, floor, log10, exp

import matplotlib.pyplot as plt

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
import sys
sys.path.insert(0, "/workspaces/richards/")

from richards.model_params import Model_Data
from richards.matrix_computer import Matrix_Computer


from richards.solver import Solver
from richards.solver_params import Solver_Data, Solver_Enum

In [3]:
K = 1000
N = 40
num_steps = 1

eps_psi_abs = 1e-7
eps_psi_rel = 0

domain_tolerance = 1 / (100 * N)

output_directory = 'output_evolutionary'

In [4]:
rho = 1000
g = pp.GRAVITY_ACCELERATION

In [5]:
theta_r = 0.026
theta_s = 0.42

alpha = 0.95

n = 2.9
K_s = 0.12

T    = 0.01

In [6]:
h_s = 0
theta_m = theta_s
m = 1 - 1/n

In [7]:
dt   = (T-0)/num_steps

In [8]:
model_data = Model_Data(theta_r=theta_r, theta_s=theta_s, alpha=alpha, n=n, K_s=K_s, T=T, num_steps=num_steps)

### Domain preparation

In [9]:
subdomain = pp.StructuredTriangleGrid([N, N], [1,1])
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

In [10]:
key = "flow"

In [11]:
def g_func(x,t): 
    return np.array([0, -1, -1])

In [12]:
def initial_pressure_func(x):
    if x[1] > 1/4 + domain_tolerance:
        return -3+x[1]
    else:
        return 1/4

In [13]:
def f(x,t):
    res = 0
    if x[1] > 1/4 + domain_tolerance:
        res = 0.006*np.cos(4/3*np.pi*(x[1]-1))*np.sin(2*np.pi*x[0])

    return res

In [14]:
for subdomain, data in mdg.subdomains(return_data=True):
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    gamma_d  = subdomain.nodes[1, :] > 1-domain_tolerance
    
    bc_essential_value = -3 * np.array(gamma_d, dtype=bool)

### Method

A quick recap:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
becames, with the time discretization:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi^{n+1}_k) {q^{n+1}_{k+1}} + \nabla \psi^{n+1}_{k+1} = -\nabla z\\
\frac{\partial \theta(\psi_k^{n+1})}{\partial \psi} \frac{\psi_{k+1}^{n+1}}{\Delta t} + \nabla \cdot {q^{n+1}_{k+1}} = \frac{\partial \theta(\psi_k^{n+1})}{\partial \psi} \frac{\psi_{k}^{n+1}}{\Delta t} + \frac{\theta (\psi^{n}) - \theta (\psi^{n+1}_k)}{\Delta t} + f^{n+1}
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

In [15]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [16]:
# assemble initial solution
cp = Matrix_Computer(mdg)

In [17]:
solver_data = Solver_Data(mdg=mdg, initial_solution=cp.P1.interpolate(subdomain, initial_pressure_func), 
                          scheme=Solver_Enum.PICARD, 
                          bc_essential = lambda t: gamma_d,
                          bc_essential_value= lambda t: bc_essential_value,
                          eps_psi_abs=eps_psi_abs, eps_psi_rel=eps_psi_rel, 
                          max_iterations_per_step=K, primal=True,
                          output_directory=output_directory, 
                          L_Scheme_value=0.2341, report_name='saturated')

solver_data.set_rhs_function_h(f)

In [18]:
solver = Solver(model_data=model_data, solver_data=solver_data)

In [19]:
start = time.time()
solver.solve()
end = time.time()

print('')
print(end - start)

Time 0.01


<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.394*(0.861784056913403*(-h + z)**2.9 + 1)**(-0.655172413793103) + 0.026,0.42], default=nan)
<lambdifygenerated-2>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.645131545005374*(-h + z)**1.9*(0.861784056913403*(-h + z)**2.9 + 1)**(-1.6551724137931),0], default=nan)


<lambdifygenerated-3>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.12*(1 - (1 - 1.0*(0.861784056913403*(-h + z)**2.9 + 1)**(-1.0))**0.655172413793103)**2*(0.861784056913403*(-h + z)**2.9 + 1)**(-0.327586206896552),0.12], default=nan)


Iteration #0001, error L2 relative psi:    0.63729543354
Iteration #0002, error L2 relative psi:    0.34263592086
Iteration #0003, error L2 relative psi:    0.07747036350
Iteration #0004, error L2 relative psi:    0.03756743358
Iteration #0005, error L2 relative psi:    0.03935246417
Iteration #0006, error L2 relative psi:    0.04530597591
Iteration #0007, error L2 relative psi:    0.05540537964


KeyboardInterrupt: 